In [2]:
import random
import pygame
from queue import PriorityQueue

random_seed = 42

pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Reading Input

In [3]:
file_path = "dataset/test/26.txt"
with open(file_path) as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

labyrinth = lines[:-2]
start_coords = lines[-2]
end_coords = lines[-1]

print("\n".join(labyrinth), start_coords, end_coords, sep='\n')

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X         X   X                 X
X XXX X   X X   XXXXX X   XXX X X
X     X X   X X   X   X X   X   X
X  XXXX   XXX XXX X  XXX X  X   X
X     X         X X         X   X
X  XX  XX XX XX X X X XXX XXXX  X
X   X     X       X             X
X X  XX     X XXX XXXXX X X X X X
X   X     X             X   X   X
XX  X X XX X XXXXXXX XX  XXX  X X
X   X               X X X   X X X
X  X XXXXXXX XX   X X X X X X   X
X               X X       X   X X
XXXX XXXX X XXX X  X  XXXXXXX X X
X         X       X   X         X
X  XXX  X X X   X XX XX   X  X  X
X X     X X   X X       X       X
X  XX   X XXX   XX X X XXXX X X X
X     X X     X             X   X
X X X  XXXXXXX  XXXXXXXX X XX X X
X   X                       X   X
XX   XXXXX X  X X X X XXXXX  XX X
X   X             X           X X
X X   XX XX XXX X X X   XX    X X
X   X       X             X     X
X    XX XXX X  XXXX XXX   XX XXXX
X X     X   X   X               X
X XXXX  X    X  X X X X XXXXXXX X
X         X   

In [4]:
def get_coords(coords_string):
    split = coords_string.split()
    x = split[1].split(sep=',')[0]
    y = split[2]
    return int(x), int(y)

In [5]:
start = get_coords(start_coords)
end = get_coords(end_coords)

print(f"start: {start}", f"end: {end}", sep='\n')

start: (29, 9)
end: (13, 15)


### Construction of the graph

In [6]:
labyrinth

['XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
 'X         X   X                 X',
 'X XXX X   X X   XXXXX X   XXX X X',
 'X     X X   X X   X   X X   X   X',
 'X  XXXX   XXX XXX X  XXX X  X   X',
 'X     X         X X         X   X',
 'X  XX  XX XX XX X X X XXX XXXX  X',
 'X   X     X       X             X',
 'X X  XX     X XXX XXXXX X X X X X',
 'X   X     X             X   X   X',
 'XX  X X XX X XXXXXXX XX  XXX  X X',
 'X   X               X X X   X X X',
 'X  X XXXXXXX XX   X X X X X X   X',
 'X               X X       X   X X',
 'XXXX XXXX X XXX X  X  XXXXXXX X X',
 'X         X       X   X         X',
 'X  XXX  X X X   X XX XX   X  X  X',
 'X X     X X   X X       X       X',
 'X  XX   X XXX   XX X X XXXX X X X',
 'X     X X     X             X   X',
 'X X X  XXXXXXX  XXXXXXXX X XX X X',
 'X   X                       X   X',
 'XX   XXXXX X  X X X X XXXXX  XX X',
 'X   X             X           X X',
 'X X   XX XX XXX X X X   XX    X X',
 'X   X       X             X     X',
 'X    XX XX

In [7]:
labyrinth_width = len(labyrinth[0])
labyrinth_height = len(labyrinth)

In [39]:
graph = {}
WALLS = []
for y in range(labyrinth_height):
    for x in range(labyrinth_width):
        if labyrinth[y][x] == "X":
            WALLS.append((x, y))
            continue
        edges = []
        for delta_x, delta_y in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            neigh_x, neigh_y = x + delta_x, y + delta_y
            if labyrinth[neigh_y][neigh_x] == " ":
                edges.append((neigh_x, neigh_y))
        graph[(x, y)] = edges

# print(f"Walls: {WALLS}") # may have long output
# print(f"Graph: {graph}") # may have long output


In [9]:
def reconstruct_path(predecessors, end):
    path = []
    pred = end
    while pred is not None:
        path.append(pred)
        if pred not in predecessors:
            pred = None
        else:
            pred = predecessors[pred]
    path.reverse()
    return path


### Setting Up Pygame For Visualization

In [10]:
CELL_SIZE = 20
FONT_SIZE = 20
FONT_COLOR = (0, 0, 0)  # black
WALL_COLOR = (0, 0, 0)  # black
START_COLOR = (0, 26, 255)  # blue
END_COLOR = (255, 0, 0)  # red
OPEN_COLOR = (0, 255, 30)  # green
CLOSED_COLOR = (0, 247, 255)  # cyan
PATH_COLOR = (255, 0, 157)  # purple
STEP_INTERVAL = 25  # in milliseconds


In [11]:
def draw_cell(surface, coords, color, sleep=True):
    if sleep:
        pygame.time.wait(STEP_INTERVAL)
    x, y = coords
    pygame.draw.rect(surface=surface, color=color,
                     rect=(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
    pygame.display.flip()

In [12]:
def run_visualization(algorithm, graph, start, end):
    random.seed(random_seed)

    pygame.init()
    screen = pygame.display.set_mode((labyrinth_width * CELL_SIZE, labyrinth_height * CELL_SIZE + 4 * FONT_SIZE))
    font = pygame.font.Font("roboto.ttf", FONT_SIZE)

    screen.fill("white")
    pygame.display.flip()

    for wall in WALLS:
        draw_cell(screen, wall, WALL_COLOR, sleep=False)
    draw_cell(screen, start, START_COLOR, sleep=False)
    draw_cell(screen, end, END_COLOR, sleep=False)

    result = algorithm(graph, start, end, screen)
    predecessors, expanded = result
    path = reconstruct_path(predecessors, end)
    for cell in path:
        draw_cell(screen, cell, PATH_COLOR)

    img_expanded = font.render(f'NODES EXPANDED: {expanded}', True, FONT_COLOR)
    screen.blit(img_expanded, (CELL_SIZE, labyrinth_height * CELL_SIZE + FONT_SIZE / 2))
    img_path = font.render(f'PATH LENGTH: {len(path) - 1}', True, FONT_COLOR)
    screen.blit(img_path, (CELL_SIZE, labyrinth_height * CELL_SIZE + 2 * FONT_SIZE))

    pygame.display.flip()

    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
    pygame.quit()

# Random Search

In [20]:
def random_search(graph, start, end, surface):
    predecessors = dict()
    opened = set()
    opened.add(start)
    expanded = set()
    while len(opened) != 0:
        current = random.choice(list(opened))
        if current == end:
            draw_cell(surface, current, CLOSED_COLOR, sleep=False)
            return predecessors, len(expanded)
        if current in graph:
            for neighbor in graph[current]:
                if neighbor not in opened | expanded:
                    opened.add(neighbor)
                    draw_cell(surface, neighbor, OPEN_COLOR)
                    predecessors[neighbor] = current
        opened.remove(current)
        expanded.add(current)
        draw_cell(surface, current, CLOSED_COLOR, sleep=False)


In [22]:
run_visualization(random_search, graph, start, end)

Here are some visualizations:

Start:

![](images/visualizations/random_search/random_search_start.png)

Almost finished:

![](images/visualizations/random_search/random_search_almost.png)

Finish:

![](images/visualizations/random_search/random_search_finish.png)

# BFS

In [17]:
def bfs_search(graph, start, end, surface):
    predecessors = dict()
    opened = set()
    opened.add(start)
    front = []
    front.append(start)
    expanded = set()
    while len(front) != 0:
        current = front.pop(0)
        if current == end:
            draw_cell(surface, current, CLOSED_COLOR, sleep=False)
            return predecessors, len(expanded)
        if current in graph:
            for neighbor in graph[current]:
                if neighbor not in opened | expanded:
                    opened.add(neighbor)
                    front.append(neighbor)
                    draw_cell(surface, neighbor, OPEN_COLOR)
                    predecessors[neighbor] = current
        opened.remove(current)
        expanded.add(current)
        draw_cell(surface, current, CLOSED_COLOR, sleep=False)

In [19]:
run_visualization(bfs_search, graph, start, end)

Here are some visualizations:

Start:

![](images/visualizations/bfs/bfs_start.png)

Almost finished:

![](images/visualizations/bfs/bfs_almost.png)

Finish:

![](images/visualizations/bfs/bfs_finish.png)

# DFS

In [25]:
def dfs_search(graph, start, end, surface):
    predecessors = dict()
    opened = set()
    opened.add(start)
    stack = []
    stack.append(start)
    expanded = set()
    while len(stack) != 0:
        current = stack.pop(-1)
        if current == end:
            draw_cell(surface, current, CLOSED_COLOR, sleep=False)
            return predecessors, len(expanded)
        if current in graph:
            for neighbor in graph[current]:
                if neighbor not in opened | expanded:
                    opened.add(neighbor)
                    stack.append(neighbor)
                    draw_cell(surface, neighbor, OPEN_COLOR)
                    predecessors[neighbor] = current
        opened.remove(current)
        expanded.add(current)
        draw_cell(surface, current, CLOSED_COLOR, sleep=False)

In [26]:
run_visualization(dfs_search, graph, start, end)

Here are some visualizations:

Start:

![](images/visualizations/dfs/dfs_start.png)

Almost finished:

![](images/visualizations/dfs/dfs_almost.png)

Finish:

![](images/visualizations/dfs/dfs_finish.png)

# Greedy Search

In [27]:
def manhattan_distance_heuristic(v1, v2):
    return abs(v1[0] - v2[0]) + abs(v1[1] - v2[1])

In [28]:
def greedy_search(graph, start, end, surface):
    predecessors = dict()
    opened = set()
    opened.add(start)
    pq = PriorityQueue()
    pq.put((manhattan_distance_heuristic(start, end), start))
    expanded = set()
    while not pq.empty():
        priority, current = pq.get()
        if current == end:
            draw_cell(surface, current, CLOSED_COLOR, sleep=False)
            return predecessors, len(expanded)
        if current in graph:
            for neighbor in graph[current]:
                if neighbor not in opened | expanded:
                    opened.add(neighbor)
                    pq.put((manhattan_distance_heuristic(neighbor, end), neighbor))
                    draw_cell(surface, neighbor, OPEN_COLOR)
                    predecessors[neighbor] = current
        opened.remove(current)
        expanded.add(current)
        draw_cell(surface, current, CLOSED_COLOR, sleep=False)

In [30]:
run_visualization(greedy_search, graph, start, end)

Here are some visualizations:

Start:

![](images/visualizations/greedy_search/greedy_search_start.png)

Almost finished:

![](images/visualizations/greedy_search/greedy_search_almost.png)

Finish:

![](images/visualizations/greedy_search/greedy_search_finish.png)

# A*

In [31]:
def a_star_search(graph, start, end, surface):
    predecessors = dict()
    distances = dict()
    distances[start] = 0
    opened = dict()
    opened[start] = manhattan_distance_heuristic(start, end) + distances[start]
    expanded = set()
    while len(opened) != 0:
        current, priority = min(opened.items(), key=lambda item: item[1])
        if current == end:
            draw_cell(surface, current, CLOSED_COLOR, sleep=False)
            return predecessors, len(expanded)
        if current in graph:
            for neighbor in graph[current]:
                if neighbor not in expanded:
                    current_dist = distances[current] + manhattan_distance_heuristic(current, neighbor)
                    if neighbor not in opened or distances[neighbor] > current_dist:
                        distances[neighbor] = current_dist
                        predecessors[neighbor] = current
                        if neighbor not in opened:
                            opened[neighbor] = manhattan_distance_heuristic(neighbor, end) + current_dist
                            draw_cell(surface, neighbor, OPEN_COLOR)
                        else:
                            opened[neighbor] = manhattan_distance_heuristic(neighbor, end) + current_dist
        opened.pop(current)
        expanded.add(current)
        draw_cell(surface, current, CLOSED_COLOR, sleep=False)

In [32]:
run_visualization(a_star_search, graph, start, end)

Here are some visualizations:

Start:

![](images/visualizations/a_star/a_star_start.png)

Almost finished:

![](images/visualizations/a_star/a_star_almost.png)

Finish:

![](images/visualizations/a_star/a_star_finish.png)